In [ ]:
import numpy as np
import pandas as pd
import googlemaps
from os import path

key = ''

In [ ]:
sourceFile = 'anagrScuoleMILANO.csv'
targetSuffix = '_geoloc'

# derive target filename
targetFile = targetSuffix.join(list(path.os.path.splitext(sourceFile)))
origData = pd.read_csv(sourceFile, sep=';', decimal = ',', index_col='CODICESCUOLA')

origData.CAPSCUOLA.replace('Non Disponibile', '', inplace=True) # fix missing CAP with empty so that geoloc works

In [ ]:
## build query addresses
queryAddresses = origData.INDIRIZZOSCUOLA.values + ', ' + origData.CAPSCUOLA.values +  ', Milano'

In [ ]:
origData.groupby('bStatale').count()

#print(len(queryAddresses))

In [ ]:
gmaps = googlemaps.Client(key=key)

nLimit = 1500

# Geocoding an address
locationsFull = pd.Series.from_array([{}]*len(queryAddresses))
# hardcoded coordinates format
coordinates = pd.DataFrame([[np.nan, np.nan]]*len(queryAddresses), columns=['Long', 'Lat'], index = origData.index)
multipleResults = pd.Series.from_array([]*len(queryAddresses))

for (i, address) in enumerate(queryAddresses):
    geocode_result = gmaps.geocode(address, components={'location':'Milano'})
    if geocode_result:
        assert isinstance(geocode_result, list) #check type
        if len(geocode_result) > 1: # multiple results, report first and
            multipleResults[i] = geocode_result
        
        # take first output
        locationsFull[i] = geocode_result[0]
            
        # parse coordinates
        thisLocation = locationsFull[i]['geometry']['location']
        coordinates['Long'][i] = thisLocation['lng']
        coordinates['Lat'][i] = thisLocation['lat']
        print(coordinates.iloc[i,:])
        
    print(i)


In [ ]:
# save full responses
import pickle
pickle_out = open(targetFile + '_results.pickle',"wb")
pickle.dump({'fullLoc': locationsFull, 'extraResults': multipleResults}, pickle_out)
pickle_out.close()

In [ ]:
for i, locat in enumerate(locationsFull):
    if not locat:
        continue
    thisLocation = locat['geometry']['location']
    coordinates['Long'][i] = thisLocation['lng']
    coordinates['Lat'][i] = thisLocation['lat']
    print(coordinates.iloc[i,:])

In [ ]:
# append coordinates
geolocData = pd.concat([origData, coordinates], axis=1)
geolocData.to_csv(targetFile, sep=';', decimal=',')

In [ ]:
# print unidentified locations
geolocData[geolocData.Lat.isnull()]